In [1]:
cd ..

/home/mehdi2277/Documents/HarveyMuddWork/Neural_Nets_Research/neural_nets_research


In [2]:
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import datetime

from neural_nets_library import training
from tree_to_sequence.tree_encoder_batch import TreeEncoderBatch
from tree_to_sequence.tree_decoder_batch import TreeDecoderBatch
from tree_to_sequence.program_datasets import *
from tree_to_sequence.tree_to_tree_attention_batch import TreeToTreeAttentionBatch
from functools import partial

In [3]:
num_vars = 10
num_ints = 11
one_hot = False
binarize_input = True
binarize_output = True
eos_token = True
long_base_case = True
input_as_seq = False
output_as_seq = False

In [4]:
for_lambda_dset = ForLambdaDataset("ANC/AdditionalForDatasets/ForWithLevels/Easy-arbitraryForList.json", binarize_input=binarize_input, 
                                   binarize_output=binarize_output, eos_token=eos_token, one_hot=one_hot, 
                                   long_base_case=long_base_case, input_as_seq=input_as_seq,
                                   output_as_seq=output_as_seq)
max_size = max([x[1].size() for x in for_lambda_dset])

In [5]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform_(param, -stdev, stdev)

In [6]:
embedding_size = 100 #... 256 is from the paper, but 100 is WAY faster
hidden_size = 256
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops)
encoder = TreeEncoderBatch(embedding_size, hidden_size)
nclass = num_vars + num_ints + len(lambda_ops)
plot_every = 100
max_num_children = 2 if binarize_output else 4

decoder = TreeDecoderBatch(embedding_size, hidden_size, max_num_children, nclass=nclass)
program_model = TreeToTreeAttentionBatch(encoder, decoder, hidden_size, embedding_size, nclass=nclass, max_size=max_size,
                                    alignment_size=alignment_size, align_type=align_type)
    
reset_all_parameters_uniform(program_model, 0.1)
encoder.initialize_forget_bias(3)
decoder.initialize_forget_bias(3)

In [7]:
# program_model = program_model.cuda()

In [8]:
optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=500, factor=0.8)

In [9]:
# Counts the number of matches between the prediction and target.
def count_matches(prediction, target):
    matches = 0
    if int(prediction.value) == int(target.value):
        matches += 1
    for i in range(min(len(target.children), len(prediction.children))):
        matches += count_matches(prediction.children[i], target.children[i])
    return matches

# Program accuracy (1 if completely correct, 0 otherwise)
def program_accuracy(prediction, target):
    if prediction.size() == count_matches(prediction, target) and \
       prediction.size() == target.size():
        return 1
    else:
        return 0

# Calculate validation accuracy (this could either be program or token accuracy)
def validation_criterion(prediction, target):
    return program_accuracy(prediction, target)

In [10]:
program_model.update_max_size(max_size)

In [11]:
start = datetime.datetime.now()
best_model, train_plot_losses, validation_plot_losses = training.yet_another_train_func(program_model, for_lambda_dset, 
                                 optimizer, lr_scheduler=None, num_epochs=1, plot_every=1,
                                 batch_size=2, print_every=99999, validation_criterion=None,
                                 use_cuda=True)
end = datetime.datetime.now()
print("TIME", end - start)

Epoch 0/0
----------


RuntimeError: Expected object of type torch.LongTensor but found type torch.FloatTensor for argument #2 'mat2'

In [ ]:
plt.plot([x * plot_every for x in range(len(train_plot_losses))], train_plot_losses)
plt.show()

plt.plot([x * plot_every for x in range(len(validation_plot_losses))], validation_plot_losses)
plt.show()

In [ ]:
train_plot_losses_old = train_plot_losses#_old + train_plot_losses_new
validation_plot_losses_old = validation_plot_losses#_old + validation_plot_losses_new

# import csv

# torch.save(program_model, "max-big-t2t-all-vars-model")
# with open("max-big-t2t-all-vars-train.txt", "w") as output:
#     writer = csv.writer(output, lineterminator='\n')
#     for val in train_plot_losses:
#         writer.writerow([val]) 
# with open("max-big-t2t-all-vars-validation.txt", "w") as output:
#     writer = csv.writer(output, lineterminator='\n')
#     for val in validation_plot_losses:
#         writer.writerow([val]) 

In [ ]:
n = num_ints + num_vars

# Check whether a node is syntactically valid, given its parent and index
# Then recursively do it for all the node's children
def check_valid(node, parent, child_index):
    category = parent_to_category_LAMBDA(parent, child_index, num_vars, num_ints)
    possible_outputs = category_to_child_LAMBDA(category, num_vars, num_ints)
    if not int(node.value) in possible_outputs:
        print("parent", parent, "child_index", child_index)
        print("ERROR", int(node.value), category)
        return False
    if (len(node.children) > 0):
        child1 = check_valid(node.children[0], int(node.value), 0)
        if not child1:
            return False
        child2 = check_valid(node.children[1], parent, child_index + 1)
        if not child2:
            return False
    return True

# Check all the programs in a dataset for syntactic accuracy
# (this is a debugging function used to double check the accuracy of your grammar)
def check_all():
    i = 0
    # Check grammar is right
    for prog in for_lambda_dset:
        correct = check_valid(prog[1], None, 0)
        if correct is False:
            print(i)
            pretty_print_tree(prog[1])
            return
        i += 1
        
        
        
check_all() #kangaroo